In [3]:
#
# All code cells are hidden in the output by default
#

# Parameter cell. Will be replaced by export_backtest_report()
parameters = {}




In [4]:
#
# Setting up
#

import traceback 
import pickle

saved_dataset = pickle.load(open(parameters["dataset_file"], "rb"))
strategy_universe = pickle.load(open(parameters["universe_file"], "rb"))


KeyError: 'dataset_file'

In [ ]:
#
# Parameters
#


import pandas as pd
from tradeexecutor.strategy.parameters import StrategyParameters
import datetime
from tradeexecutor.strategy.default_routing_options import TradeRouting
from tradeexecutor.strategy.cycle import CycleDuration
from tradingstrategy.timebucket import TimeBucket


class Parameters:
    id = "equally-weighted-index"
    
    if saved_dataset.set.time_bucket >= TimeBucket.d1:
        # For longer timeframesd cycle and time bucket are always the same
        candle_time_bucket = saved_dataset.set.time_bucket
        cycle_duration = CycleDuration.from_timebucket(saved_dataset.set.time_bucket)
    else:
        # Handle hourly specially with 4h cycles
        candle_time_bucket = saved_dataset.set.time_bucket
        cycle_duration = CycleDuration.cycle_4h

    # Coingecko categories to include
    # s
    # See list here: TODO
    #
    chain_id = saved_dataset.set.chain
    exchanges = saved_dataset.set.exchanges

    #
    # Basket construction and rebalance parameters
    #
    min_asset_universe = 5  # How many assets we need in the asset universe to start running the index
    max_assets_in_portfolio = 10  # How many assets our basket can hold once
    allocation = 0.98  # Allocate all cash to volatile pairs
    # min_rebalance_trade_threshold_pct = 0.05  # % of portfolio composition must change before triggering rebalacne
    individual_rebalance_min_threshold_usd = 75.0  # Don't make buys less than this amount
    per_position_cap_of_pool = 0.01  # Never own more than % of the lit liquidity of the trading pool
    max_concentration = 0.20  # How large % can one asset be in a portfolio once
    min_portfolio_weight = 0.0050  # Close position / do not open if weight is less than 50 BPS

    # For the length of trailing sharpe used in inclusion criteria
    rolling_volume_bars = pd.Timedelta("7d") // candle_time_bucket.to_timedelta()
    rolling_volatility_bars = pd.Timedelta("7d") // candle_time_bucket.to_timedelta()
    tvl_ewm_span = 7 * 24  # Smooth TVL inclusin criteria
    min_volume = saved_dataset.set.min_weekly_volume  # USD
    min_tvl = saved_dataset.set.min_tvl
    min_token_sniffer_score = 20  # 20 = AAVE

    #
    #
    # Backtesting only
    # Limiting factor: Aave v3 on Base starts at the end of DEC 2023
    #
    backtest_start = saved_dataset.set.start
    backtest_end = saved_dataset.set.end
    initial_cash = 100_000

    #
    # Live only
    #
    routing = TradeRouting.default
    # required_history_period = datetime.timedelta(days=2 * 14 + 1)
    slippage_tolerance = 0.0060  # 0.6%

parameters = StrategyParameters.from_class(Parameters)

# Dataset information

Information about the prepared backtest dataset.

In [ ]:
df = saved_dataset.get_info()
display(df)

# Equally-weighed index backtest parameters

These parameters present the trading strategy parameters to generate equally-weighted index for this dataset.

In [ ]:
from tradeexecutor.strategy.parameters import display_parameters
display_parameters(parameters)

# Prepare benchmark

- Run a backtest for equally weighted index strategy
- Used to display 

In [ ]:
from tradingstrategy.utils.forward_fill import forward_fill
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.state.identifier import TradingPairIdentifier
import pandas as pd


from tradeexecutor.strategy.pandas_trader.indicator import IndicatorSource
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.strategy.pandas_trader.indicator import calculate_and_load_indicators_inline
from tradeexecutor.strategy.pandas_trader.indicator import IndicatorDependencyResolver
from tradeexecutor.state.types import USDollarAmount
from tradeexecutor.strategy.pandas_trader.indicator_decorator import IndicatorRegistry

indicators = IndicatorRegistry()

empty_series = pd.Series([], index=pd.DatetimeIndex([]))

@indicators.define()
def volatility(close: pd.Series, rolling_volatility_bars: int) -> pd.Series:
    """Calculate the rolling volatility for rebalancing the index for each decision cycle."""
    price_diff = close.pct_change()
    rolling_std = price_diff.rolling(window=rolling_volatility_bars).std()
    return rolling_std


@indicators.define()
def rolling_cumulative_volume(volume: pd.Series, rolling_volume_bars: int) -> pd.Series:
    """Calculate rolling volume of the pair.

    - Used in inclusion criteria
    """
    rolling_volume = volume.rolling(window=rolling_volume_bars).sum()
    return rolling_volume


@indicators.define()
def rolling_liquidity_avg(close: pd.Series, rolling_volume_bars: int) -> pd.Series:
    """Calculate rolling liquidity average

    - This is either TVL or XY liquidity (one sided) depending on the trading pair DEX type

    - Used in inclusion criteria
    """
    rolling_liquidity_close = close.rolling(window=rolling_volume_bars).mean()
    return rolling_liquidity_close


@indicators.define(dependencies=(rolling_cumulative_volume,), source=IndicatorSource.strategy_universe)
def volume_inclusion_criteria(
        strategy_universe: TradingStrategyUniverse,
        min_volume: USDollarAmount,
        rolling_volume_bars: int,
        dependency_resolver: IndicatorDependencyResolver,
) -> pd.Series:
    """Calculate pair volume inclusion criteria.

    - Avoid including illiquid / broken pairs in the set: Pair is included when it has enough volume

    TODO: Add liquidity check later

    :return:
        Series where each timestamp is a list of pair ids meeting the criteria at that timestamp
    """

    series = dependency_resolver.get_indicator_data_pairs_combined(
        rolling_cumulative_volume,
        parameters={"rolling_volume_bars": rolling_volume_bars},
    )

    # Get mask for days when the rolling volume meets out criteria
    mask = series >= min_volume
    mask_true_values_only = mask[mask == True]

    # Turn to a series of lists
    series = mask_true_values_only.groupby(level='timestamp').apply(lambda x: x.index.get_level_values('pair_id').tolist())
    return series


@indicators.define(source=IndicatorSource.tvl)
def tvl(
    close: pd.Series,
    execution_context: ExecutionContext,
    timestamp: datetime.datetime,
    candle_time_bucket: TimeBucket,
) -> pd.Series:
    """Get TVL series for a pair.

    - Because TVL data is 1d and we use 1h everywhere else, we need to forward fill

    - Use previous hourly close as the value
    """
    if execution_context.live_trading:
        # TVL is daily data.
        # We need to forward fill until the current hour.
        # Use our special ff function.
        assert isinstance(timestamp, pd.Timestamp), f"Live trading needs forward-fill end time, we got {timestamp}"
        df = pd.DataFrame({"close": close})
        df_ff = forward_fill(
            df,
            Parameters.candle_time_bucket.to_frequency(),
            columns=("close",),
            forward_fill_until=pd.Timestamp(timestamp),
        )
        series = df_ff["close"]
        return series
    else:        
        return close.resample(candle_time_bucket.to_frequency()).ffill()
        # return close


@indicators.define(dependencies=(tvl,), source=IndicatorSource.dependencies_only_per_pair)
def tvl_ewm(
    pair: TradingPairIdentifier,
    tvl_ewm_span: float,
    dependency_resolver: IndicatorDependencyResolver,
) -> pd.Series:
    """Get smoothed TVL series for a pair.

    - Interpretation: If you set span=5, for example, the ewm function will compute an exponential moving average where the weight of the most recent observation is about 33.3% (since α=2/(5+1)≈0.333) and this weight decreases exponentially for older observations.

    - We forward fill gaps, so there is no missing data in decide_trades()

    - Currently unused in the strategy itself
    """
    tvl_ff = dependency_resolver.get_indicator_data(
        tvl,
        pair=pair,
    )
    return tvl_ff.ewm(span=tvl_ewm_span).mean()


@indicators.define(dependencies=(tvl_ewm, tvl), source=IndicatorSource.dependencies_only_universe)
def tvl_inclusion_criteria(
        min_tvl: USDollarAmount,
        dependency_resolver: IndicatorDependencyResolver,
) -> pd.Series:
    """The pair must have min XX,XXX USD one-sided TVL to be included.

    - If the Uniswap pool does not have enough ETH or USDC deposited, skip the pair as a scam

    :return:
        Series where each timestamp is a list of pair ids meeting the criteria at that timestamp
    """

    series = dependency_resolver.get_indicator_data_pairs_combined(tvl)
    mask = series >= min_tvl
    # Turn to a series of lists
    mask_true_values_only = mask[mask == True]
    series = mask_true_values_only.groupby(level='timestamp').apply(lambda x: x.index.get_level_values('pair_id').tolist())
    return series


@indicators.define(
    source=IndicatorSource.strategy_universe
)
def trading_availability_criteria(
        strategy_universe: TradingStrategyUniverse,
) -> pd.Series:
    """Is pair tradeable at each hour.

    - The pair has a price candle at that
    - Mitigates very corner case issues that TVL/liquidity data is per-day whileas price data is natively per 1h
      and the strategy inclusion criteria may include pair too early hour based on TVL only,
      leading to a failed attempt to rebalance in a backtest
    - Only relevant for backtesting issues if we make an unlucky trade on the starting date
      of trading pair listing

    :return:
        Series with with index (timestamp) and values (list of pair ids trading at that hour)
    """
    # Trading pair availability is defined if there is a open candle in the index for it.
    # Because candle data is forward filled, we should not have any gaps in the index.
    candle_series = strategy_universe.data_universe.candles.df["open"]
    pairs_per_timestamp = candle_series.groupby(level='timestamp').apply(lambda x: x.index.get_level_values('pair_id').tolist())
    return pairs_per_timestamp


@indicators.define(
    dependencies=[
        volume_inclusion_criteria,
        tvl_inclusion_criteria,
        trading_availability_criteria
    ],
    source=IndicatorSource.strategy_universe
)
def inclusion_criteria(
    strategy_universe: TradingStrategyUniverse,
    min_volume: USDollarAmount,
    rolling_volume_bars: int,
    min_tvl: USDollarAmount,
    dependency_resolver: IndicatorDependencyResolver
) -> pd.Series:
    """Pairs meeting all of our inclusion criteria.

    - Give the tradeable pair set for each timestamp

    :return:
        Series where index is timestamp and each cell is a list of pair ids matching our inclusion criteria at that moment
    """

    volume_series = dependency_resolver.get_indicator_data(
        volume_inclusion_criteria,
        parameters={
            "min_volume": min_volume,
            "rolling_volume_bars": rolling_volume_bars,
        },
    )

    tvl_series = dependency_resolver.get_indicator_data(
        tvl_inclusion_criteria,
        parameters={
            "min_tvl": min_tvl,
        },
    )

    trading_availability_series = dependency_resolver.get_indicator_data(trading_availability_criteria)

    #
    # Process all pair ids as a set and the final inclusion
    # criteria is union of all sub-criterias
    #

    df = pd.DataFrame({
        "tvl_pair_ids": tvl_series,
        "volume_pair_ids": volume_series,
        "trading_availability_pair_ids": trading_availability_series,
    })

    # https://stackoverflow.com/questions/33199193/how-to-fill-dataframe-nan-values-with-empty-list-in-pandas
    df = df.fillna("").apply(list)

    def _combine_criteria(row):
        final_set = set(row["volume_pair_ids"]) & \
                    set(row["tvl_pair_ids"]) & \
                    set(row["trading_availability_pair_ids"])
        return final_set

    union_criteria = df.apply(_combine_criteria, axis=1)

    # Inclusion criteria data can be spotty at the beginning when there is only 0 or 1 pairs trading,
    # so we need to fill gaps to 0
    full_index = pd.date_range(
        start=union_criteria.index.min(),
        end=union_criteria.index.max(),
        freq=Parameters.candle_time_bucket.to_frequency(),
    )
    reindexed = union_criteria.reindex(full_index, fill_value=[])
    return reindexed


@indicators.define(dependencies=(volume_inclusion_criteria,), source=IndicatorSource.dependencies_only_universe)
def volume_included_pair_count(
        min_volume: USDollarAmount,
        rolling_volume_bars: int,
        dependency_resolver: IndicatorDependencyResolver
) -> pd.Series:
    series = dependency_resolver.get_indicator_data(
        volume_inclusion_criteria,
        parameters={"min_volume": min_volume, "rolling_volume_bars": rolling_volume_bars},
    )
    return series.apply(len)


@indicators.define(dependencies=(tvl_inclusion_criteria,), source=IndicatorSource.dependencies_only_universe)
def tvl_included_pair_count(
        min_tvl: USDollarAmount,
        dependency_resolver: IndicatorDependencyResolver
) -> pd.Series:
    """Calculate number of pairs in meeting volatility criteria on each timestamp"""
    series = dependency_resolver.get_indicator_data(
        tvl_inclusion_criteria,
        parameters={"min_tvl": min_tvl},
    )
    series = series.apply(len)

    # TVL data can be spotty at the beginning when there is only 0 or 1 pairs trading,
    # so we need to fill gaps to 0
    full_index = pd.date_range(
        start=series.index.min(),
        end=series.index.max(),
        freq=Parameters.candle_time_bucket.to_frequency(),
    )
    # Reindex and fill NaN with zeros
    reindexed = series.reindex(full_index, fill_value=0)
    return reindexed


@indicators.define(dependencies=(inclusion_criteria,), source=IndicatorSource.dependencies_only_universe)
def all_criteria_included_pair_count(
        min_volume: USDollarAmount,
        min_tvl: USDollarAmount,
        rolling_volume_bars: int,
        dependency_resolver: IndicatorDependencyResolver
) -> pd.Series:
    """Series where each timestamp is the list of pairs meeting all inclusion criteria.

    :return:
        Series with pair count for each timestamp
    """
    series = dependency_resolver.get_indicator_data(
        "inclusion_criteria",
        parameters={
            "min_volume": min_volume,
            "min_tvl": min_tvl,
            "rolling_volume_bars": rolling_volume_bars,
        },
    )
    return series.apply(len)


@indicators.define(source=IndicatorSource.strategy_universe)
def trading_pair_count(
    strategy_universe: TradingStrategyUniverse,
) -> pd.Series:
    """Get number of pairs that trade at each timestamp.

    - Pair must have had at least one candle before the timestamp to be included

    - Exclude benchmarks pairs we do not trade

    :return:
        Series with pair count for each timestamp
    """

    # Get pair_id, timestamp -> timestamp, pair_id index    
    series = strategy_universe.data_universe.candles.df["open"]
    swap_index = series.index.swaplevel(0, 1)

    seen_pairs = set()
    seen_data = {}

    for timestamp, pair_id in swap_index:
        seen_pairs.add(pair_id)
        seen_data[timestamp] = len(seen_pairs)

    series = pd.Series(seen_data.values(), index=list(seen_data.keys()))
    return series


# Calculate all indicators where parameters have changed and store the result on disk
indicator_data = calculate_and_load_indicators_inline(
    strategy_universe=strategy_universe,
    create_indicators=indicators.create_indicators,
    parameters=parameters,
    verbose=False,
)




In [ ]:
from tradeexecutor.strategy.alpha_model import AlphaModel
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pandas_trader.strategy_input import StrategyInput
from tradeexecutor.backtest.backtest_runner import run_backtest_inline
from tradeexecutor.strategy.tvl_size_risk import USDTVLSizeRiskModel
from tradeexecutor.strategy.weighting import weight_passthrouh, weight_equal


def decide_trades(
    input: StrategyInput
) -> list[TradeExecution]:
    """For each strategy tick, generate the list of trades."""
    parameters = input.parameters
    position_manager = input.get_position_manager()
    state = input.state
    timestamp = input.timestamp
    indicators = input.indicators
    strategy_universe = input.strategy_universe

    # Build signals for each pair 
    alpha_model = AlphaModel(
        timestamp,
        close_position_weight_epsilon=parameters.min_portfolio_weight,  # 10 BPS is our min portfolio weight
    )
    
    # Get pairs included in this rebalance cycle.
    # This includes pair that have been pre-cleared in inclusion_criteria()
    # with volume, volatility and TVL filters 
    included_pairs = indicators.get_indicator_value(
        "inclusion_criteria",
        na_conversion=False,
    )
    if included_pairs is None:
        included_pairs = []
    
    for pair_id in included_pairs:
        pair = strategy_universe.get_pair_by_id(pair_id)
        weight = 1
        
        alpha_model.set_signal(
            pair,
            weight,
        )

    # Calculate how much dollar value we want each individual position to be on this strategy cycle,
    # based on our total available equity
    portfolio = position_manager.get_current_portfolio()
    portfolio_target_value = portfolio.get_total_equity() * parameters.allocation

    # Select max_assets_in_portfolio assets in which we are going to invest
    # Calculate a weight for ecah asset in the portfolio using 1/N method based on the raw signal
    alpha_model.select_top_signals(count=parameters.max_assets_in_portfolio)
    alpha_model.assign_weights(method=weight_equal)
    # alpha_model.assign_weights(method=weight_by_1_slash_n)

    #
    # Normalise weights and cap the positions
    # 
    size_risk_model = USDTVLSizeRiskModel(
        pricing_model=input.pricing_model,
        per_position_cap=parameters.per_position_cap_of_pool,  # This is how much % by all pool TVL we can allocate for a position
        missing_tvl_placeholder_usd=0,  # Placeholder for missing TVL data until we get the data off the chain
    )

    alpha_model.normalise_weights(
        investable_equity=portfolio_target_value,
        size_risk_model=size_risk_model,
        max_weight=parameters.max_concentration,
    )

    # Load in old weight for each trading pair signal,
    # so we can calculate the adjustment trade size
    alpha_model.update_old_weights(
        state.portfolio,
        ignore_credit=True,
    )
    alpha_model.calculate_target_positions(position_manager)

    # Shift portfolio from current positions to target positions
    # determined by the alpha signals (momentum)
    
    # rebalance_threshold_usd = portfolio_target_value * parameters.min_rebalance_trade_threshold_pct
    rebalance_threshold_usd = parameters.individual_rebalance_min_threshold_usd
    
    assert rebalance_threshold_usd > 0.1, "Safety check tripped - something like wrong with strat code"
    trades = alpha_model.generate_rebalance_trades_and_triggers(
        position_manager,
        min_trade_threshold=rebalance_threshold_usd,  # Don't bother with trades under XXXX USD
        invidiual_rebalance_min_threshold=parameters.individual_rebalance_min_threshold_usd,
        execution_context=input.execution_context,
    )
    return trades  # Return the list of trades we made in this cycle


state = None
try:
    result = run_backtest_inline(
        name=parameters.id,
        engine_version="0.5",
        decide_trades=decide_trades,
        create_indicators=indicators.create_indicators,
        client=None,
        universe=strategy_universe,
        parameters=parameters,
        # log_level=logging.INFO,
        max_workers=1,
        start_at=parameters.backtest_start,
        end_at=parameters.backtest_end,
    )
    
    state = result.state
    
    trade_count = len(list(state.portfolio.get_all_trades()))
    print(f"Backtesting completed, backtested strategy made {trade_count} trades")
except Exception as e:
    print(f"Failed to run backtest: {e}")
    traceback.print_exc()

# Asset universe

- Show how the asset universe evolves over time
- Respect the minimum TVL and volume criteria from dataset labelling 

In [ ]:
import plotly.express as px

df = pd.DataFrame({
    "Inclusion criteria met (all)": indicator_data.get_indicator_series("all_criteria_included_pair_count"),
    f"Volume criteria {Parameters.min_volume:,} USD / {Parameters.rolling_volume_bars * Parameters.candle_time_bucket.to_pandas_timedelta() / datetime.timedelta(hours=24)} days met": indicator_data.get_indicator_series("volume_included_pair_count"),
    f"TVL criteria {Parameters.min_tvl:,} USD met": indicator_data.get_indicator_series("tvl_included_pair_count"),
    "Visible pairs": indicator_data.get_indicator_series("trading_pair_count"),
})

try:
    fig = px.line(df, title='Trading pairs available for strategy to trade')
    fig.update_yaxes(title="Number of assets")
    fig.update_xaxes(title="Time")
    fig.show()
except Exception as e: 
    print(f"Could not render asset universe chart: {e}")
    import traceback
    traceback.print_exc()


Available trading pairs at the end of the period

In [ ]:
display(df.tail(25))

# Equally-weighed index

This is the [benchmark](https://tradingstrategy.ai/glossary/benchmark) of this trading universe.

Here we compare display the results 

- Equally-weighted index of the trading universe
- Some notable crypto assets in this trading universe 



In [ ]:
from tradeexecutor.analysis.multi_asset_benchmark import get_benchmark_data
from tradeexecutor.visual.benchmark import visualise_equity_curve_benchmark

if state:
    start_at = state.get_trading_time_range()[0]
    
    benchmark_indexes = get_benchmark_data(
        strategy_universe,
        cumulative_with_initial_cash=state.portfolio.get_initial_cash(),
        start_at=start_at,
    )
    
    fig = visualise_equity_curve_benchmark(
        name="This strategy",
        title="Strategy vs. benchmark assets",
        state=state,
        all_cash=state.portfolio.get_initial_cash(),
        benchmark_indexes=benchmark_indexes,
        height=800,
        log_y=False,
        start_at=start_at,
    )
    
    fig.show()

# Equity curve

The equity curve allows to examine how stable the strategy profitability is.

Here we plot

- The strategy equity curve
- Maximum drawdown
- Daily profit

In [ ]:
import pandas as pd
from tradeexecutor.analysis.multi_asset_benchmark import get_benchmark_data
from tradeexecutor.visual.benchmark import visualise_equity_curve_benchmark

if state:
    benchmark_indexes = get_benchmark_data(
        strategy_universe,
        cumulative_with_initial_cash=state.portfolio.get_initial_cash(),
        max_count=4,
        # start_at=state.get_trading_time_range()[0],
    )
    
    fig = visualise_equity_curve_benchmark(
        state=state,    
        benchmark_indexes=benchmark_indexes,
        height=800,
        log_y=True,
    )
    
    fig.show()
else:
    print("No backtest results availabl4e")

## Equity curve with drawdown

- Showing drawndown as a separate chart

In [ ]:
from tradeexecutor.visual.equity_curve import calculate_equity_curve, calculate_returns
from tradeexecutor.visual.equity_curve import visualise_equity_curve

if state:
    curve = calculate_equity_curve(state)
    returns = calculate_returns(curve)
    fig = visualise_equity_curve(returns)
    display(fig)
else:
    returns = None

# Market performance metrics

Side-by-side comparison of equally-weighted index and some benchmark adjusts.

See [risk-adjusted return](https://tradingstrategy.ai/glossary/risk-adjusted-return) to learn more about how to compare risk and reward ratios of different trading strategies.


In [ ]:
from tradeexecutor.analysis.multi_asset_benchmark import compare_strategy_backtest_to_multiple_assets

if state:
    compare_strategy_backtest_to_multiple_assets(
        state,
        strategy_universe,
        display=True,
    )
else:
    print("No backtest results availabl4e")

# Monthly returns

Here we show the backtested returns by each month, and visualise the streaks of good and bad months.


In [ ]:
from tradeexecutor.visual.equity_curve import visualise_returns_over_time

if returns is not None:
    visualise_returns_over_time(returns)
else:
    print("No backtest results availabl4e")

# Trading metrics

Overview of the performance of trades this strategy took.

- How many winning and losing trades we had
- How much on average each trade made

In [ ]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

if state:
    analysis = build_trade_analysis(state.portfolio)
    
    try:
        summary_metrics = analysis.calculate_summary_statistics(state=state, time_bucket=strategy_universe.data_universe.time_bucket)
        summary_metrics.display()
    except Exception as e:
        print(f"Summary metrics information cannot be displayed. Exception: {e}")
        traceback.print_exc()
else:
    print("No backtest results availabl4e")

# Periodic return distribution

Show performance variations for different timeframes.

In [ ]:
from tradeexecutor.visual.equity_curve import visualise_returns_distribution

if returns is not None:
    visualise_returns_distribution(returns)
else:
    print("No backtest results availabl4e")

# Asset weights

- Show weights of different assets during the backtest period.
- Equally weighted index considers liquidity of the trading pair (TVL)
  and may not be able to deploy all capital

In [ ]:
from tradeexecutor.analysis.weights import calculate_asset_weights, visualise_weights

if state:
    weights_series = calculate_asset_weights(state)
    
    fig = visualise_weights(
        weights_series,
        normalised=False,
        include_reserves=True,
        template="plotly_white"
    )
    fig.show()
else:
    print("No backtest results availabl4e")

# Trading pair breakdown

Show gains and losses in individual trading pairs.

In [ ]:
from tradeexecutor.analysis.multipair import analyse_multipair
from tradeexecutor.analysis.multipair import format_multipair_summary

if state and strategy_universe.get_pair_count() > 1:
    multipair_summary = analyse_multipair(state)
    display(format_multipair_summary(multipair_summary))
else:
    print("This strategy traded only a single trading pair or no trades")